# 加载数据

In [1]:
# import warnings
# warnings.filterwarnings("ignore")
# import pandas as pd
# from sklearn.metrics import accuracy_score
# from tqdm import tqdm
# import torch
# import torch.nn as nn
# from transformers import BertModel, BertTokenizer, BertForMaskedLM
# import torch.nn.functional as F

# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing
# from transformers import BertTokenizer
# from transformers import AdamW, get_linear_schedule_with_warmup
# from torch.utils.data import Dataset, DataLoader,RandomSampler,DistributedSampler
# import os
# from random import shuffle

In [2]:
# batch_size = 16
# max_length = 512


# def load_data(path):
#     data = []
#     with open(path, encoding="utf-8") as f:
#         lines = f.readlines()
#         context = ""
#         for line in lines:
#             if line != '\n':
#                 context += line
#             elif context != "":
#                 data.append(context)
#                 context = ""
#             else:
#                 continue
#         if context != "":
#             data.append(context)
#             context = ""
#     return data
    
# # 加载数据
# # df = load_data('./data.txt')

# data_frame = pd.read_csv('../Toshuoge/test_node_output/test_corpus_data.csv', encoding='utf-8')


# df = data_frame['context'].to_list()
# len(df)




In [3]:
# # 数据处理，把你的无标签文本数据转化为MLM任务的数据
# def preprocess_mlm_data(text_data, tokenizer, max_length):
#     inputs = tokenizer(text_data, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
#     inputs["labels"] = inputs.input_ids.detach().clone()
#     rand = torch.rand(inputs.input_ids.shape)
#     mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

#     for i in range(inputs.input_ids.size(0)):
#         selection = torch.flatten(mask_arr[i].nonzero()).tolist()
#         inputs.input_ids[i, selection] = tokenizer.mask_token_id

#     return inputs

# model_name = "bert-base-uncased"
# #封装为Dataloader
# tokenizer = BertTokenizer.from_pretrained(model_name)
# # text_data是一个包含的无标签文本数据的列表
# inputs = preprocess_mlm_data(df, tokenizer, max_length)
# dataset = torch.utils.data.TensorDataset(inputs["input_ids"], inputs["attention_mask"], inputs["labels"])
# train_sampler = RandomSampler(dataset)
# dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=batch_size)

# 初始化模型

In [4]:
# model = BertForMaskedLM.from_pretrained("bert-base-uncased")  # "bert-large-uncased"

# # 解冻所有层
# for param in model.bert.parameters():
#     param.requires_grad = True

# # 将embedding层的requires_grad属性设为False
# for name, param in model.named_parameters():
#     if name.startswith('bert.embeddings'):
#         param.requires_grad = False    

# device=torch.device("cpu")
        
# model.to(device)

In [5]:
# num_epochs = 10 #300

# lr_base = 5e-5
# weight_decay = 0.01
# accumulation_steps = batch_size*2 #梯度累加

# optimizer = AdamW(model.parameters(),  lr=lr_base,betas=(0.9, 0.999), weight_decay=weight_decay) 

# num_examples = len(df)
# num_training_steps = (num_examples // batch_size) * num_epochs // accumulation_steps
# num_warmup_steps = int(num_training_steps * 0.1)

# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) #注意，这里的Warm up长短是按照step来的，每跑一个mini-batch就是一个step

In [6]:
# from torch.utils.tensorboard import SummaryWriter 
# writer = SummaryWriter('tf-logs') 

# # 训练循环
# num_epochs = 10 #300
# j=1
# model.train()
# for epoch in range(num_epochs):
#     for i,batch in enumerate(dataloader):
#         input_ids, attention_mask, labels = [x.to(device) for x in batch]
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss = loss/accumulation_steps
#         loss.backward()

#         if((i+1)%accumulation_steps)==0:
#             writer.add_scalar("Lr/step",optimizer.param_groups[0]['lr'],j)
#             j+=1
#             optimizer.step()        # 反向传播，更新网络参数
#             optimizer.zero_grad()   # 清空梯度
#             scheduler.step()
#     torch.cuda.empty_cache()
#     writer.add_scalar("Loss/train", loss.item(), epoch+1)
#     writer.add_scalar("Lr/train",optimizer.param_groups[0]['lr'],epoch+1)
#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

#     num_groups = len(optimizer.param_groups)
#     print(num_groups)
#     for i in range(0, num_groups):  #num_groups-5
#         param_group = optimizer.param_groups[i]
#         layer_lr = param_group['lr']
#         print("Layer {} learning rate: {}".format(i, layer_lr))


# # 保存fine-tuned模型
# state_dict = {"model":model.state_dict(),"optimizer":optimizer.state_dict(),"epoch":epoch+1}
# name = f"withintrain/model_withintrain_{epoch}.pth"#存放最终模型的路径
# torch.save(state_dict,name)
# writer.close()

In [44]:
# !pip install -q transformers
# !pip install -q datasets

import multiprocessing
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers

from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer, AutoConfig
from transformers import BertForMaskedLM, DistilBertForMaskedLM
from transformers import BertTokenizer, DistilBertTokenizer
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from tokenizers import BertWordPieceTokenizer

In [45]:
# HYPERPARAMS
SEED_SPLIT = 0
SEED_TRAIN = 0

MAX_SEQ_LEN = 128
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5 
LR_WARMUP_STEPS = 100
WEIGHT_DECAY = 0.01

In [46]:
 # load data
dtf_mlm = pd.read_csv('../Toshuoge/test_node_output/test_corpus_data.csv', encoding='utf-8')
#dtf_mlm = dtf_mlm.rename(columns={"review_content": "text"})

# Train/Valid Split
df_train, df_valid = train_test_split(
    dtf_mlm, test_size=0.15, random_state=SEED_SPLIT
)

len(df_train), len(df_valid)

# Convert to Dataset object
train_dataset = Dataset.from_pandas(df_train[['text']].dropna())
valid_dataset = Dataset.from_pandas(df_valid[['text']].dropna())

In [47]:
print(len(df_train), len(df_valid))

385 69


In [48]:
'''
bert-base-uncased  # 12-layer, 768-hidden, 12-heads, 109M parameters
distilbert-base-uncased  # 6-layer, 768-hidden, 12-heads, 65M parameters
'''

MODEL = 'bert'
bert_type = 'bert-base-uncased'

if MODEL == 'distilbert':
    TokenizerClass = DistilBertTokenizer 
    ModelClass = DistilBertForMaskedLM 
elif MODEL == 'bert':
    TokenizerClass = BertTokenizer
    ModelClass = BertForMaskedLM 
elif MODEL == 'roberta':
    TokenizerClass = RobertaTokenizer
    ModelClass = RobertaForMaskedLM
elif MODEL == 'scibert':
    TokenizerClass = AutoTokenizer
    ModelClass = AutoModelForMaskedLM

# bert_type = '/home/wshuo/dukto/uncased_L-12_H-768_A-12'
# TokenizerClass = BertTokenizer
# ModelClass = BertForMaskedLM
    
tokenizer = TokenizerClass.from_pretrained(
            bert_type, use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
            )
model = ModelClass.from_pretrained(bert_type)

loading file vocab.txt from cache at /home/wshuo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/a265f773a47193eed794233aa2a0f0bb6d3eaa63/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/wshuo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/a265f773a47193eed794233aa2a0f0bb6d3eaa63/tokenizer_config.json
loading configuration file config.json from cache at /home/wshuo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/a265f773a47193eed794233aa2a0f0bb6d3eaa63/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-

In [49]:
# test creating new tokenizer
tokenizer = TokenizerClass.from_pretrained(
            bert_type, use_fast=True, do_lower_case=False, max_len=MAX_SEQ_LEN
            )
print(tokenizer.tokenize('ros'))
print(tokenizer.tokenize('rplidar'))




new_tokens = []
df_region_word_example = pd.read_csv('../Toshuoge/test_region_word_in_examples.csv', encoding='utf-8') 

example_wds = df_region_word_example['word'].dropna().tolist()

# print(len(example_wds))

new_tokens.extend(example_wds)


df_region_word_final = pd.read_csv('../Toshuoge/region_word_frequency_final.csv', encoding='utf-8') 

final_wds = df_region_word_final['word'].dropna().tolist()
for wd in final_wds:
    if wd not in new_tokens:
        new_tokens.append(wd)




print(len(new_tokens))


# print(new_tokens)


loading file vocab.txt from cache at /home/wshuo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/a265f773a47193eed794233aa2a0f0bb6d3eaa63/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/wshuo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/a265f773a47193eed794233aa2a0f0bb6d3eaa63/tokenizer_config.json
loading configuration file config.json from cache at /home/wshuo/.cache/huggingface/hub/models--bert-base-uncased/snapshots/a265f773a47193eed794233aa2a0f0bb6d3eaa63/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-

['ro', '##s']
['r', '##pl', '##ida', '##r']
4671


In [51]:
num_added_tokens = tokenizer.add_tokens(new_tokens[:850]) 

print(num_added_tokens)

0


In [52]:
model.resize_token_embeddings(len(tokenizer)) 









print(tokenizer.tokenize('ros'))
print(tokenizer.tokenize('rplidar'))





tokenizer.save_pretrained("./new_tokenizer")



vocab = open('./new_tokenizer/vocab.txt', 'r')
vocab_wds = [line.strip('\n') for line in vocab]




phrase = "ros stands for robot operating system, which is an open-source framework for building robotic software."
print(phrase)
tokens = tokenizer.encode_plus(phrase, add_special_tokens=True, return_tensors='pt')
#     print(tokens['input_ids'])
ids = tokens['input_ids'].numpy().tolist()
print(ids)

id_char = ""
for i in ids[0]:
    if i>= len(vocab_wds):
        id_char += "{0}({1}), ".format(i, "@@@")
    else:
        id_char += "{0}({1}), ".format(i, vocab_wds[i])
print(id_char)

tokenizer config file saved in ./new_tokenizer/tokenizer_config.json
Special tokens file saved in ./new_tokenizer/special_tokens_map.json
added tokens file saved in ./new_tokenizer/added_tokens.json


['ros']
['rplidar']
ros stands for robot operating system, which is an open-source framework for building robotic software.
[[101, 30522, 4832, 2005, 8957, 6728, 2121, 31096, 12835, 2291, 1010, 2029, 2003, 2019, 31002, 7705, 2005, 2311, 20478, 4007, 1012, 102]]
101([CLS]), 30522(@@@), 4832(stands), 2005(for), 8957(robot), 6728(op), 2121(##er), 31096(@@@), 12835(ng), 2291(system), 1010(,), 2029(which), 2003(is), 2019(an), 31002(@@@), 7705(framework), 2005(for), 2311(building), 20478(robotic), 4007(software), 1012(.), 102([SEP]), 


In [53]:
def tokenize_function(row):
    return tokenizer(
        row['text'],
        padding='max_length',
        truncation=True,
        max_length=MAX_SEQ_LEN,
        return_special_tokens_mask=True)
  
column_names = train_dataset.column_names

train_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=column_names,
)

valid_dataset = valid_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=multiprocessing.cpu_count(),
    remove_columns=column_names,
)

Map (num_proc=16):   0%|          | 0/385 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/69 [00:00<?, ? examples/s]

In [54]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

steps_per_epoch = int(len(train_dataset) / TRAIN_BATCH_SIZE)

# logging_steps=1,
training_args = TrainingArguments(
    output_dir='./bert-news',
    logging_dir='./LMlogs',             
    num_train_epochs=100,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    warmup_steps=LR_WARMUP_STEPS,
    save_steps=steps_per_epoch,
    save_total_limit=3,
    weight_decay=WEIGHT_DECAY,
    learning_rate=LEARNING_RATE, 
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='loss', 
    greater_is_better=False,
    seed=SEED_TRAIN,
    logging_steps=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model("./model") #save your custom model

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/home/wshuo/anaconda3/envs/torch/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 385
  Num Epochs = 100
  Instantaneous batch size per device = 64
  To

Epoch,Training Loss,Validation Loss
1,3.505100,3.752929
2,1.441700,3.310165
3,6.545800,3.520307
4,0.258300,3.236142
5,4.533400,3.540601
6,0.914300,3.461921
7,0.484600,3.238062
8,2.762000,2.819933
9,2.692600,2.716869
10,2.846100,2.846476


The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-7
Configuration saved in ./bert-news/checkpoint-7/config.json
Model weights saved in ./bert-news/checkpoint-7/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-7/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-7/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-7/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-322] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not e

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-77
Configuration saved in ./bert-news/checkpoint-77/config.json
Model weights saved in ./bert-news/checkpoint-77/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-77/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-77/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-77/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-56] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are 

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-147
Configuration saved in ./bert-news/checkpoint-147/config.json
Model weights saved in ./bert-news/checkpoint-147/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-147/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-147/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-147/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-126] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_ma

...

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-427
Configuration saved in ./bert-news/checkpoint-427/config.json
Model weights saved in ./bert-news/checkpoint-427/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-427/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-427/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-427/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-413] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_ma

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-497
Configuration saved in ./bert-news/checkpoint-497/config.json
Model weights saved in ./bert-news/checkpoint-497/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-497/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-497/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-497/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-483] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_ma

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-567
Configuration saved in ./bert-news/checkpoint-567/config.json
Model weights saved in ./bert-news/checkpoint-567/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-567/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-567/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-567/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-546] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_ma

The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 69
  Batch size = 64
Saving model checkpoint to ./bert-news/checkpoint-637
Configuration saved in ./bert-news/checkpoint-637/config.json
Model weights saved in ./bert-news/checkpoint-637/pytorch_model.bin
tokenizer config file saved in ./bert-news/checkpoint-637/tokenizer_config.json
Special tokens file saved in ./bert-news/checkpoint-637/special_tokens_map.json
added tokens file saved in ./bert-news/checkpoint-637/added_tokens.json
Deleting older checkpoint [bert-news/checkpoint-623] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_ma



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./bert-news/checkpoint-602 (score: 0.9780388474464417).
Saving model checkpoint to ./model
Configuration saved in ./model/config.json
Model weights saved in ./model/pytorch_model.bin
tokenizer config file saved in ./model/tokenizer_config.json
Special tokens file saved in ./model/special_tokens_map.json
added tokens file saved in ./model/added_tokens.json


In [19]:
phrase = "ROS stands for Robot Operating System, which is an open-source framework for building robotic Software."
print(phrase)
tokens = tokenizer.encode_plus(phrase, add_special_tokens=True, return_tensors='pt')
#     print(tokens['input_ids'])
ids = tokens['input_ids'].numpy().tolist()
print(ids)


vocab = open('./model/vocab.txt', 'r')
vocab_wds = [line.strip('\n') for line in vocab]

id_char = ""
for i in ids[0]:
    id_char += "{0}({1}), ".format(i, vocab_wds[i])
print(id_char)

ros stands for Robot Operating System, which is an open-source framework for building robotic Software.
[[101, 20996, 2015, 4832, 2005, 100, 100, 100, 1010, 2029, 2003, 2019, 2330, 1011, 3120, 7705, 2005, 2311, 20478, 100, 1012, 102]]
101([CLS]), 20996(ro), 2015(##s), 4832(stands), 2005(for), 100([UNK]), 100([UNK]), 100([UNK]), 1010(,), 2029(which), 2003(is), 2019(an), 2330(open), 1011(-), 3120(source), 7705(framework), 2005(for), 2311(building), 20478(robotic), 100([UNK]), 1012(.), 102([SEP]), 
